## Générer des dossiers de nouvelles actifs

In [2]:
import os
import pandas as pd

# Définir les dossiers d'entrée et de sortie
dossier_entree = "new actions csv"  # Dossier contenant les fichiers CSV
dossier_sortie = "new actions"  # Dossier pour enregistrer les fichiers XLSX

# Vérifier si le dossier de sortie existe, sinon le créer
if not os.path.exists(dossier_sortie):
    os.makedirs(dossier_sortie)

# Parcourir tous les fichiers CSV dans le dossier d'entrée
for fichier in os.listdir(dossier_entree):
    if fichier.endswith(".csv"):  # Vérifier que le fichier est un CSV
        chemin_fichier = os.path.join(dossier_entree, fichier)
        
        try:
            # Lire le fichier CSV en tant que texte brut
            df = pd.read_csv(chemin_fichier, dtype=str)
            df.columns = df.columns.str.strip()  # Supprimer les espaces des noms de colonnes

            # Extraire le nom du fichier sans extension pour l'utiliser comme "Actif"
            nom_actif = os.path.splitext(fichier)[0]

            # Dictionnaire de correspondance des noms de colonnes
            correspondance_colonnes = {
                "Date": "Dates",
                "Ouv.": "Ouverture",
                "Plus Haut": "Max",
                "Plus Bas": "Min",
                "Dernier": "Fermeture",
                "Vol.": "Volume"
            }

            # Renommer les colonnes existantes si elles sont présentes dans le fichier
            colonnes_valides = {ancien: nouveau for ancien, nouveau in correspondance_colonnes.items() if ancien in df.columns}
            df = df.rename(columns=colonnes_valides)

            # Conserver uniquement les colonnes souhaitées
            colonnes_finales = ["Dates", "Ouverture", "Max", "Min", "Fermeture", "Volume"]
            df = df.reindex(columns=colonnes_finales)

            # Ajouter la colonne "Actif"
            df.insert(0, "Actif", nom_actif)

            # Convertir la colonne "Dates" au format datetime et trier par ordre chronologique
            df["Dates"] = pd.to_datetime(df["Dates"], format="%d/%m/%Y", errors="coerce")

            # Reformater les dates pour ne garder que "JJ/MM/AAAA"
            df["Dates"] = df["Dates"].dt.strftime("%d/%m/%Y")
            df = df.sort_values(by="Dates")

            # Convertir les colonnes numériques en float (en remplaçant les virgules par des points)
            colonnes_numeriques = ["Ouverture", "Max", "Min", "Fermeture"]
            for col in colonnes_numeriques:
                df[col] = df[col].str.replace(",", ".", regex=True).astype(float)

            # Convertir la colonne "Volume" en valeurs numériques
            def convertir_volume(volume):
                if isinstance(volume, str):
                    volume = volume.replace(",", ".")  # Remplacer la virgule par un point
                    if "K" in volume:
                        return float(volume.replace("K", "")) * 1000  # Convertir en milliers
                    elif "M" in volume:
                        return float(volume.replace("M", "")) * 1000000  # Convertir en millions
                return pd.to_numeric(volume, errors="coerce")  # Autres cas, conversion en numérique

            df["Volume"] = df["Volume"].apply(convertir_volume)

            # Générer le chemin de sortie avec le format .xlsx (Excel)
            fichier_sortie = os.path.join(dossier_sortie, f"{nom_actif}.xlsx")

            # Enregistrer les données dans un fichier Excel sans l'en-tête
            with pd.ExcelWriter(fichier_sortie, engine="openpyxl") as writer:
                df.to_excel(writer, sheet_name="Actions Data", index=False, header=False)

            print(f"Conversion terminée : {fichier} → {fichier_sortie} (sans en-tête)")

        except Exception as e:
            print(f"Erreur lors de la lecture du fichier {fichier} : {e}")


Conversion terminée : Generali.csv → new actions/Generali.xlsx (sans en-tête)
Conversion terminée : Sanofi.csv → new actions/Sanofi.xlsx (sans en-tête)
Conversion terminée : AirBus.csv → new actions/AirBus.xlsx (sans en-tête)
Conversion terminée : SchneiderElectric.csv → new actions/SchneiderElectric.xlsx (sans en-tête)
Conversion terminée : Bouygues.csv → new actions/Bouygues.xlsx (sans en-tête)


## Générer un dossier de CAC40

In [5]:
# traitement de Cac40.xlsx
df=pd.read_excel("Cac40.xlsx")
# Convertir la colonne "Dates" au format datetime et trier par ordre chronologique
df = df.sort_values(by="Dates")
df["Dates"] = pd.to_datetime(df["Dates"], format="%d/%m/%Y", errors="coerce")
# Reformater les dates pour ne garder que "JJ/MM/AAAA"
df["Dates"] = df["Dates"].dt.strftime("%d/%m/%Y")

df.to_excel("new actions/Cac40.xlsx",index=False,header=False)


